In [ ]:
'''
In the assignment you have to emulate typical CRUD operations using plain functions.

CRUD: Create, Read (Retrieve), Update, Delete.
Firstly, implement parsing function.
For database emulation we can use list, i.e.,

DATABASE = list(), can be global variable of passed into each function as
              argument.
'''

In [ ]:
import hashlib
from datetime import datetime

In [ ]:
USER_SCHEME = ("id", "first_name", "second_name", "email", "password")
RECORD_SCHEME = ("id", "date", "content", "user", "title")
ENTITIES = ('User', 'Record')
DATABASE = []

In [ ]:
def _get_entity(s: str) -> str:
  """Gets named entity ('User' or 'Record') from string.

  Args:
    s(str): string with User or Record entity.

  Returns:
    string: extracted entity.
  """
  if ENTITIES[0] in s:
    return ENTITIES[0]
  elif ENTITIES[1] in s:
    return ENTITIES[1]
  else:
    return ""

In [ ]:
def _get_content(scheme: tuple, s: str) -> dict:
  result = {}
  for key in scheme:
    if key + "=" in s:
      print("Found!")
      idx = s.index(key + "=")
      print(f"Index of first character of {key + '='} is {idx}")
      search_index = idx + len(key + '=')
      print(f"Index of the last character of {key + '='} is {search_index}")
      i = search_index
      sub_string = ""

      while i < len(s) - 1 and s[i] not in {',', ')'}:
              sub_string += s[i]
              i += 1
      if not sub_string:
          while s[i] != ')' and i != len(s) - 1:
              i += 1

      result[key] = sub_string
      print(result)
    else:
      result[key] = None
  return result

In [ ]:
# testing user
_get_content(scheme=USER_SCHEME, s=users[2])

In [ ]:
# testing record
_get_content(scheme=RECORD_SCHEME, s=records[2])

In [ ]:
def parse_string(string: str) -> dict:
  """Each of functions below should parse input string.
  Function must identify entety: record or user.
  Implement function to parse standard input string based on example:

  record: "Record(id=1, date=26.02.2004, content=Some example,
                   user=1, title=Example title)"
  user:    "User(id=1, first_name=test name, second_name=test surname,
                 email=test@test.test, password=123)"
  Args:
    string (str): represents entity like examples.

  Return:
    dict: {'record': {"id": 1, "date": datetime.datetime, 'content':
                      'Some example',"user": 1, 'title': "Example title"}}
          or
          {'user': {"id": 1, 'first_name': 'test_name', ...,
                    "password": hash(password)}}

  """
  entity = _get_entity(s=string)
  if entity:
        scheme = USER_SCHEME if entity == 'User' else RECORD_SCHEME
        result_dict = _get_content(scheme=scheme, s=string)
        if entity == 'User' and 'password' in result_dict:
            password = result_dict['password']
            if password is not None:
                result_dict['password'] = hashlib.sha256(password.encode()).hexdigest()
        if 'id' not in result_dict:
            print("ID not found in the input string.")
            return {}
        return {entity.lower(): result_dict}
  else:
      return {}

In [ ]:
parse_string("User(id=1, first_name=test name, second_name=test surname,email=test@test.test, password=123)")

In [ ]:
parse_string("Record(id=1, date=26.02.2004, content=Some example #1, user=1, title=Example title)")

In [ ]:
def create_user(user: str) -> dict:
  """
  Performs creation operation. Input: string with record. Output: parsed string
  converted into dict (json).

  Args:
    user (str): "User(id=1, first_name=test name, second_name=test surname,
                 email=test@test.test, password=123)"
  Returns:
    None

  dict: user = {"id": 1, 'first_name': 'test_name', ...,
                  "password": hash(password) }
  DATABASE.append(dict)

  """
  result = parse_string(user)
  user_id = result['user']['id']

  for existing_user in DATABASE:
      if 'user' in existing_user and existing_user['user']['id'] == user_id:
          existing_user['user'].update(result['user'])
          if 'password' in existing_user['user']:
              password = existing_user['user']['password']
              if password is not None:
                  existing_user['user']['password'] = hashlib.sha256(password.encode()).hexdigest()
          return existing_user

  if 'password' in result['user']:
      password = result['user']['password']
      if password is not None:
          result['user']['password'] = hashlib.sha256(password.encode()).hexdigest()
  DATABASE.append(result)
  return dict

In [ ]:
def create_record(record: str) -> dict:
  """
  Performs creation operation. Input: string with record. Output: parsed string
  converted into dict (json).

  Args:
    record (str): "Record(id=1, date=26.02.2004, content=Some example,
                   user=1, 'title'=Example title)"
  Returns:
    dict: record = {"id": 1, "date": datetime.datetime, 'content': 'Some example',
                    "user": 1, 'title': "Example title"}

  # parse string and get record in dict, record = ...
  # DATABASE.append(record)

  """
  result = parse_string(record)
  record_id = result['record']['id']

  for record in DATABASE:
      if 'record' in record and record['record']['id'] == record_id:
          record['record'].update(result['record'])
          return record
  return None

  DATABASE.append(result)
  return dict()

In [ ]:
create_user("User(id=1, first_name=test name, second_name=test surname, email=test@test.test, password=123)")
create_user("User(id=2, first_name=another test name, second_name=another test surname, email=test1@test.test, password=456)")
create_user("User(id=3, first_name=test name x1, second_name=test surname x2, email=test123@test.test)")
create_user("User(id=4, first_name=, second_name=test surname x3, email=train@test.test)")
create_user("User(id=5, first_name=test nameee, second_name=test surnameee, email=test@tesdt.test, password=12309)")

In [ ]:
create_record("Record(id=1, date=26.02.2004, content=Some example #1, user=1, title=Example title)")
create_record("Record(id=2, date=01.10.2013, content=Some example #2, user=3, title=Example title one)")
create_record("Record(id=3, date=12.13.2008, content=Some example #3, user=2, title=Example title of user 2)")

In [ ]:
create_record("Record(id=8, date=26.02.2004, content=Some example #1, user=6, title=Example title)")

In [ ]:
def update_record(record: str) -> dict:
  """
  Performs update operation. Input: string with record. Output: parsed string
  converted into dict (json).

  Args:
    record (str): "Record(id=1, date=24.02.2004, content=Some example,
                   user=1, 'title'=Example title)"
  Returns:
    dict: record = {"id": 1, "date": datetime.datetime, 'content': 'Some example',
                    "user": 1, 'title': "Example title"}

  """
  parsed_record = parse_string(record)
  record_id = parsed_record.get('record', {}).get('id')
  print(f'\nSearching for record with ID: {record_id}')

  for record in DATABASE:
      if 'record' in record and record['record'].get('id') == record_id:
          record['record'].update(parsed_record['record'])
          return record['record']
  return print(f'Record with ID={record_id} not found.')

In [ ]:
update_record("Record(id=3, date=24.02.2004, content=Some updated example, user=1, title=Updated title)")

In [ ]:
update_record("Record(id=10, date=24.02.2004, content=Some updated example, user=1, title=Updated title)")

In [ ]:
def update_user(user: str) -> dict:
    """
    Performs update operation on a user record. Input: string with user record.
    Output: parsed string converted into dict (json).

    Args:
        user (str): "User(id=1, first_name=test name, second_name=test surname,
                     email=test@test.test, password=123)"
    Returns:
        dict: Updated user record if found, else None.
    """
    parsed_user = parse_string(user)
    user_id = parsed_user.get('user', {}).get('id')
    print(f'\nSearching for user with ID: {user_id}')

    if not user_id:
        print("User ID not found.")
        return None

    for user_entry in DATABASE:
        if 'user' in user_entry and user_entry['user'].get('id') == user_id:
            user_entry['user'].update(parsed_user['user'])
            if 'password' in user_entry['user']:
                password = user_entry['user']['password']
                if password is not None:
                    user_entry['user']['password'] = hashlib.sha256(password.encode()).hexdigest()
            return user_entry['user']
    return print(f'Record with ID={user_id} not found.')

In [ ]:
def read_record(record_id: int or str or None = None, record_content: str = None) -> dict:
    """
    Returns record by given content. Search DATABASE, find and return record.

    Args:
        record_content (string): record content.

    Returns:
        dict: record = {"id": 1, "date": datetime.datetime, 'content': 'Some example',
                        "user": 1, 'title': "Example title"}

    """

    if record_id is not None:
        record_id = str(record_id)
        for record in DATABASE:
            if 'record' in record and record['record'].get('id') == record_id:
                return record['record']
        else:
            return None
    else:
        for record in DATABASE:
            if 'record' in record:
                record_data = record['record']
                if (record_data.get('date') == record_content or
                    record_data.get('content') == record_content or
                    record_data.get('user') == record_content or
                    record_data.get('title') == record_content):
                    return record_data
        else:
            print('Not found record')
            return None

In [ ]:
read_record(record_id=2)

In [ ]:
read_record(record_content='26.02.2004')

In [ ]:
def read_user(user_id: int or str or dict or None = None, first_name: str = None, second_name: str = None, email: str = None) -> dict:
    """
    Returns user by given id or other attributes. Search DATABASE, find and return user.

    Args:
        user_id (int or str or dict): user id or user dict.
        first_name (str): user first name.
        second_name (str): user second name.
        email (str): user email.

    Returns:
        dict: user = {"id": 1, 'first_name': 'test_name', ..., "password": hash(password) }

    """
    if user_id is not None:
        if isinstance(user_id, dict):
            user = user_id
        else:
            for user in DATABASE:
                if 'user' in user and user['user'].get('id') == user_id:
                    return user['user']
            else:
                return None
    else:
        for user in DATABASE:
            if 'user' in user:
                user_data = user['user']
                if (user_data.get('first_name') == first_name or
                    user_data.get('second_name') == second_name or
                    user_data.get('email') == email):
                    return user_data
        else:
            print('Not found user')
            return None

In [ ]:
read_user(first_name='UPDATED test name x1')

In [ ]:
def delete_record(record_id: str) -> dict:
    """
    Removes a record by its ID. Searches the DATABASE, finds the record with the given ID, and deletes it.

    Args:
        record_id (int): The ID of the record to delete.

    Returns:
        dict: The deleted record.
    """
    for i, entity in enumerate(DATABASE):
        if entity.get('record') and entity['record'].get('id') == record_id:
            del DATABASE[i]
            return entity['record']
    return None

In [ ]:
delete_record(record_id='2')

In [ ]:
def delete_user(user_id: str) -> dict:
    """
    Removes a user by its ID. Searches the DATABASE, finds the user with the given ID, and deletes it.

    Args:
        user_id (int): The ID of the user to delete.

    Returns:
        dict: The deleted user.
    """
    for i, entity in enumerate(DATABASE):
        if entity.get('user') and entity['user'].get('id') == user_id:
            del DATABASE[i]
            return entity['user']
    return None